In [ ]:
# Importar las librerías necesarias
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score
)

# diccionario para guardar todas las métricas
metricas = {}

In [2]:
# --- 1. Carga del Conjunto de Datos ---
data = pd.read_csv("../data/subset/clean_subset_lifestyledata_rows5200_seed5200.csv")

# --- 2. Codificación de Etiquetas (Label Encoding) ---
label_encoder = LabelEncoder()
# Se transforma la variable objetivo 'Workout_Type' a valores numéricos.
data['Workout_Type'] = label_encoder.fit_transform(data['Workout_Type'])

# --- 3. Codificación One-Hot (One-Hot Encoding) ---
# Se define la lista de columnas categóricas nominales a transformar.
nominal_cols = ['Gender']
# sparse_output=False: Devuelve una matriz densa (array de NumPy) en lugar de una dispersa.
# handle_unknown='ignore': Si aparece una categoría no vista durante la transformación, la ignora.
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Esto crea nuevas columnas binarias para cada categoría.
encoded = ohe.fit_transform(data[nominal_cols])
# Se convierte la matriz resultante en un DataFrame con nombres de columna apropiados.
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(nominal_cols))

# --- 4. Combinación de los Datos Procesados ---
# Se elimina la columna original 'Gender' del DataFrame principal.
# reset_index(drop=True) asegura que los índices se alineen correctamente para la concatenación.
data = data.drop(columns=nominal_cols).reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Se concatenan el DataFrame original y el nuevo DataFrame con las columnas codificadas.
# axis=1 indica que la unión se realiza por columnas.
data = pd.concat([data, encoded_df], axis=1)

# --- 5. Visualización ---
data.head()

,Age,Weight_kg,Height_m,Max_BPM,Avg_BPM,Resting_BPM,Session_Duration_hours,Calories_Burned,Workout_Type,Gender_Female,Gender_Male
0,21.14,101.05,1.95,171.17,130.81,68.96,0.97,959.43,2,0.0,1.0
1,44.17,41.63,1.78,167.33,158.46,63.95,1.48,1424.35,0,0.0,1.0
2,20.07,63.81,1.78,187.86,137.11,60.93,1.70,1766.64,0,1.0,0.0
3,36.30,59.77,1.78,183.83,120.32,60.01,0.85,1028.50,1,1.0,0.0
4,51.99,57.60,1.56,166.25,151.82,67.97,1.66,1295.80,3,0.0,1.0


# Árbol de Decisiones:

#### 1. Árbol de Decisiones - CC:SI - ED:NO - Outliers:NO - Balanceo: NO

In [ ]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_1 = data.copy()
X = data_tree_1.drop("Workout_Type", axis=1)
y = data_tree_1["Workout_Type"]

# ================================================================
# 🔁 Tres muestras
# ================================================================
random_states = [111, 222, 333]  # tres seeds diferentes
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # --- Modelo 1: Árbol con Gini ---
    modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=seed)
    modelo_gini.fit(X_train, y_train)
    y_pred_gini = modelo_gini.predict(X_test)

    # --- Modelo 2: Árbol con Entropía ---
    modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=seed)
    modelo_entropy.fit(X_train, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test)

    # --- Modelo 3: Entropía con poda ---
    modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=seed)
    modelo_entropy_pruned.fit(X_train, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

    # --- Calcular métricas ---
    modelos = {
        "Gini": y_pred_gini,
        "Entropía": y_pred_entropy,
        "Entropía Podado": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        
        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })
        
        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")



=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini
Accuracy: 0.7327, Precision: 0.7412, Recall: 0.7327, F1-Score: 0.7350

Modelo: Entropía
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

Modelo: Entropía Podado
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini
Accuracy: 0.7452, Precision: 0.7431, Recall: 0.7452, F1-Score: 0.7438

Modelo: Entropía
Accuracy: 0.7875, Precision: 0.7968, Recall: 0.7875, F1-Score: 0.7892

Modelo: Entropía Podado
Accuracy: 0.7875, Precision: 0.7968, Recall: 0.7875, F1-Score: 0.7892

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini
Accuracy: 0.7587, Precision: 0.7538, Recall: 0.7587, F1-Score: 0.7559

Modelo: Entropía
Accuracy: 0.7837, Precision: 0.8018, Recall: 0.7837, F1-Score: 0.7823

Modelo: Entropía Podado
Accuracy: 0.7837, Precision: 0.8018, Recall: 0.7837, F1-Score: 0.7823


In [4]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 2. Árbol de Decisiones - CC:SI - ED:NO - Outliers:NO - Balanceo: SI

In [32]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_2 = data.copy()
X = data_tree_2.drop("Workout_Type", axis=1)
y = data_tree_2["Workout_Type"]

# ================================================================
# 🔁 Tres muestras con class_weight='balanced'
# ================================================================
random_states = [111, 222, 333]  # Tres seeds diferentes
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # Dividir datos (80/20 estratificado)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # --- Modelo 1: Árbol con Gini (class_weight='balanced') ---
    modelo_gini = DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_gini.fit(X_train, y_train)
    y_pred_gini = modelo_gini.predict(X_test)

    # --- Modelo 2: Árbol con Entropía (class_weight='balanced') ---
    modelo_entropy = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_entropy.fit(X_train, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test)

    # --- Modelo 3: Entropía con poda (class_weight='balanced') ---
    modelo_entropy_pruned = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        min_samples_split=5, # Poda
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_entropy_pruned.fit(X_train, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

    # --- Calcular métricas ---
    modelos = {
        "Gini_Balanced": y_pred_gini,
        "Entropía_Balanced": y_pred_entropy,
        "Entropía_Podado_Balanced": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas, usando zero_division=0 para un manejo robusto
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")


=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Balanced
Accuracy: 0.7327, Precision: 0.7412, Recall: 0.7327, F1-Score: 0.7350

Modelo: Entropía_Balanced
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

Modelo: Entropía_Podado_Balanced
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Balanced
Accuracy: 0.7375, Precision: 0.7338, Recall: 0.7375, F1-Score: 0.7348

Modelo: Entropía_Balanced
Accuracy: 0.7740, Precision: 0.7726, Recall: 0.7740, F1-Score: 0.7720

Modelo: Entropía_Podado_Balanced
Accuracy: 0.7740, Precision: 0.7726, Recall: 0.7740, F1-Score: 0.7720

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Balanced
Accuracy: 0.7365, Precision: 0.7361, Recall: 0.7365, F1-Score: 0.7243

Modelo: Entropía_Balanced
Accuracy: 0.7846, Precision: 0.8026, Recall: 0.7846, F1-Score: 0.7833

Modelo: Entropía_Podado_Balanced

In [6]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 3. Árbol de Decisiones - CC:SI - ED:NO - Outliers:SI - Balanceo: NO

In [33]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_3 = data.copy()

# seleccionar solo las columnas numéricas
num_cols = data_tree_3.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_tree_3[num_cols].quantile(0.25)
Q3 = data_tree_3[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_tree_3[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_tree_3[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_tree_3[mask].reset_index(drop=True)

print("Tamaño original:", data_tree_3.shape)
print("Tamaño sin outliers:", data_clean.shape)

# Separar X e y con datos limpios
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# ================================================================
# 🔁 Tres muestras
# ================================================================
random_states = [111, 222, 333]  
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # --- Modelo 1: Árbol con Gini ---
    modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=seed)
    modelo_gini.fit(X_train, y_train)
    y_pred_gini = modelo_gini.predict(X_test)

    # --- Modelo 2: Árbol con Entropía ---
    modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=seed)
    modelo_entropy.fit(X_train, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test)

    # --- Modelo 3: Entropía con poda ---
    modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=seed)
    modelo_entropy_pruned.fit(X_train, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

    # --- Calcular métricas ---
    modelos = {
        "Gini_Clean": y_pred_gini,
        "Entropía_Clean": y_pred_entropy,
        "Entropía_Podado_Clean": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")


Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Clean
Accuracy: 0.7399, Precision: 0.7479, Recall: 0.7399, F1-Score: 0.7403

Modelo: Entropía_Clean
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

Modelo: Entropía_Podado_Clean
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Clean
Accuracy: 0.7933, Precision: 0.8097, Recall: 0.7933, F1-Score: 0.7919

Modelo: Entropía_Clean
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

Modelo: Entropía_Podado_Clean
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Clean
Accuracy: 0.7923, Precision: 0.7979, Recall: 0.7923, F1-Score: 0.7928

Modelo: Entropía_Clean
Accuracy: 0.8200, Precision: 0.8163, Recall: 0.8200, F1-Score: 0.81

In [8]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 4. Árbol de Decisiones - CC:SI - ED:NO - Outliers:SI - Balanceo: SI

In [34]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_4 = data.copy()

# seleccionar solo las columnas numéricas
num_cols = data_tree_4.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_tree_4[num_cols].quantile(0.25)
Q3 = data_tree_4[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_tree_4[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_tree_4[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_tree_4[mask].reset_index(drop=True)

print("Tamaño original:", data_tree_4.shape)
print("Tamaño sin outliers:", data_clean.shape)

# Separar X e y con datos limpios
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]  #
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # --- Modelo 1: Árbol con Gini + Balanced ---
    modelo_gini = DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_gini.fit(X_train, y_train)
    y_pred_gini = modelo_gini.predict(X_test)

    # --- Modelo 2: Árbol con Entropía + Balanced ---
    modelo_entropy = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        random_state=seed,
        class_weight='balanced'
    )
    modelo_entropy.fit(X_train, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test)

    # --- Modelo 3: Entropía con poda + Balanced ---
    modelo_entropy_pruned = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        min_samples_split=5, # Poda
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_entropy_pruned.fit(X_train, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

    # --- Calcular métricas ---
    modelos = {
        "Gini_Clean_Balanced": y_pred_gini,
        "Entropía_Clean_Balanced": y_pred_entropy,
        "Entropía_Podado_Clean_Balanced": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Clean_Balanced
Accuracy: 0.7745, Precision: 0.7935, Recall: 0.7745, F1-Score: 0.7710

Modelo: Entropía_Clean_Balanced
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

Modelo: Entropía_Podado_Clean_Balanced
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Clean_Balanced
Accuracy: 0.7933, Precision: 0.8097, Recall: 0.7933, F1-Score: 0.7919

Modelo: Entropía_Clean_Balanced
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

Modelo: Entropía_Podado_Clean_Balanced
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Clean_Balanced
Accuracy: 0.7943, Precision: 0.7994, Recall: 0.7943, F1-Score: 0.7946

Modelo: Entropía_Clean_Bala

In [10]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 5. Árbol de Decisiones - CC:SI - ED:SI - Outliers:NO - Balanceo: NO

In [35]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_5 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
X = data_tree_5.drop("Workout_Type", axis=1)
y = data_tree_5["Workout_Type"]

# Definir el escalador
scaler = StandardScaler()

# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]  
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # 1. Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 2. Preprocesamiento: Escalado solo de columnas numéricas (excluyendo 'Gender')
    numeric_cols = [col for col in X_train.columns if col not in ['Gender']] 

    # Hacer copias para el escalado
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    # Escalar
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

    # 3. Entrenamiento y Predicción de Modelos
    
    # --- Modelo 1: Árbol con Gini ---
    modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=seed)
    modelo_gini.fit(X_train_scaled, y_train)
    y_pred_gini = modelo_gini.predict(X_test_scaled)

    # --- Modelo 2: Árbol con Entropía ---
    modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=seed)
    modelo_entropy.fit(X_train_scaled, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test_scaled)

    # --- Modelo 3: Entropía con poda ---
    modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=seed)
    modelo_entropy_pruned.fit(X_train_scaled, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

    # 4. Calcular métricas
    modelos = {
        "Gini_Scaled": y_pred_gini,
        "Entropía_Scaled": y_pred_entropy,
        "Entropía_Podado_Scaled": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")


=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Scaled
Accuracy: 0.7327, Precision: 0.7412, Recall: 0.7327, F1-Score: 0.7350

Modelo: Entropía_Scaled
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

Modelo: Entropía_Podado_Scaled
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Scaled
Accuracy: 0.7452, Precision: 0.7431, Recall: 0.7452, F1-Score: 0.7438

Modelo: Entropía_Scaled
Accuracy: 0.7875, Precision: 0.7968, Recall: 0.7875, F1-Score: 0.7892

Modelo: Entropía_Podado_Scaled
Accuracy: 0.7875, Precision: 0.7968, Recall: 0.7875, F1-Score: 0.7892

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Scaled
Accuracy: 0.7587, Precision: 0.7538, Recall: 0.7587, F1-Score: 0.7559

Modelo: Entropía_Scaled
Accuracy: 0.7837, Precision: 0.8018, Recall: 0.7837, F1-Score: 0.7823

Modelo: Entropía_Podado_Scaled
Accuracy: 0.7837,

In [12]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 6. Árbol de Decisiones - CC:SI - ED:SI - Outliers:NO - Balanceo: SI

In [36]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_6 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
X = data_tree_6.drop("Workout_Type", axis=1)
y = data_tree_6["Workout_Type"]

# Definir el escalador
scaler = StandardScaler()

# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]  
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # 1. Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 2. Preprocesamiento: Escalado solo de columnas numéricas
    numeric_cols = [col for col in X_train.columns if col not in ['Gender']] # Se asume 'Gender' es la única no numérica relevante aquí

    # Hacer copias para el escalado
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    # Escalar
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

    # 3. Entrenamiento y Predicción de Modelos (con class_weight='balanced')

    # --- Modelo 1: Árbol con Gini + Balanced ---
    modelo_gini = DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_gini.fit(X_train_scaled, y_train)
    y_pred_gini = modelo_gini.predict(X_test_scaled)

    # --- Modelo 2: Árbol con Entropía + Balanced ---
    modelo_entropy = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_entropy.fit(X_train_scaled, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test_scaled)

    # --- Modelo 3: Entropía con poda + Balanced ---
    modelo_entropy_pruned = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        min_samples_split=5, # Poda
        random_state=seed,
        class_weight='balanced' 
    )
    modelo_entropy_pruned.fit(X_train_scaled, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

    # 4. Calcular métricas
    modelos = {
        "Gini_Scaled_Balanced": y_pred_gini,
        "Entropía_Scaled_Balanced": y_pred_entropy,
        "Entropía_Podado_Scaled_Balanced": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")


=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Scaled_Balanced
Accuracy: 0.7327, Precision: 0.7412, Recall: 0.7327, F1-Score: 0.7350

Modelo: Entropía_Scaled_Balanced
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

Modelo: Entropía_Podado_Scaled_Balanced
Accuracy: 0.7779, Precision: 0.7923, Recall: 0.7779, F1-Score: 0.7804

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Scaled_Balanced
Accuracy: 0.7375, Precision: 0.7338, Recall: 0.7375, F1-Score: 0.7348

Modelo: Entropía_Scaled_Balanced
Accuracy: 0.7740, Precision: 0.7726, Recall: 0.7740, F1-Score: 0.7720

Modelo: Entropía_Podado_Scaled_Balanced
Accuracy: 0.7740, Precision: 0.7726, Recall: 0.7740, F1-Score: 0.7720

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Scaled_Balanced
Accuracy: 0.7365, Precision: 0.7361, Recall: 0.7365, F1-Score: 0.7243

Modelo: Entropía_Scaled_Balanced
Accuracy: 0.7846, Precision: 0.8026, Recall: 0.

In [14]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 7. Árbol de Decisiones - CC:SI - ED:SI - Outliers:SI - Balanceo: NO

In [37]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_7 = data.copy()

# seleccionar solo las columnas numéricas
num_cols = data_tree_7.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_tree_7[num_cols].quantile(0.25)
Q3 = data_tree_7[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_tree_7[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_tree_7[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_tree_7[mask].reset_index(drop=True)

print("Tamaño original:", data_tree_7.shape)
print("Tamaño sin outliers:", data_clean.shape)

# Separar X e y con datos limpios
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Definir el escalador
scaler = StandardScaler()
# Definir columnas a escalar (asumiendo 'Gender' es la única no numérica en X)
numeric_cols = [col for col in X.columns if col not in ['Gender']]

# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]  # Tres seeds diferentes
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # 1. Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 2. Preprocesamiento: Escalado
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    # Ajustar y transformar solo las columnas numéricas
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])


    # 3. Entrenamiento y Predicción de Modelos

    # --- Modelo 1: Árbol con Gini ---
    modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=seed)
    modelo_gini.fit(X_train_scaled, y_train)
    y_pred_gini = modelo_gini.predict(X_test_scaled)

    # --- Modelo 2: Árbol con Entropía ---
    modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=seed)
    modelo_entropy.fit(X_train_scaled, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test_scaled)

    # --- Modelo 3: Entropía con poda ---
    modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=seed)
    modelo_entropy_pruned.fit(X_train_scaled, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

    # 4. Calcular métricas
    modelos = {
        "Gini_Clean_Scaled": y_pred_gini,
        "Entropía_Clean_Scaled": y_pred_entropy,
        "Entropía_Podado_Clean_Scaled": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Clean_Scaled
Accuracy: 0.7399, Precision: 0.7479, Recall: 0.7399, F1-Score: 0.7403

Modelo: Entropía_Clean_Scaled
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

Modelo: Entropía_Podado_Clean_Scaled
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Clean_Scaled
Accuracy: 0.7933, Precision: 0.8097, Recall: 0.7933, F1-Score: 0.7919

Modelo: Entropía_Clean_Scaled
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

Modelo: Entropía_Podado_Clean_Scaled
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Clean_Scaled
Accuracy: 0.7923, Precision: 0.7979, Recall: 0.7923, F1-Score: 0.7928

Modelo: Entropía_Clean_Scaled
Accuracy: 0

In [16]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

#### 8. Árbol de Decisiones - CC:SI - ED:SI - Outliers:SI - Balanceo: SI

In [38]:
# ================================================================
# 📂 Datos base
# ================================================================
data_tree_8 = data.copy()

# 1. Eliminar outliers (IQR)
num_cols = data_tree_8.select_dtypes(include=['float64', 'int64']).columns
Q1 = data_tree_8[num_cols].quantile(0.25)
Q3 = data_tree_8[num_cols].quantile(0.75)
IQR = Q3 - Q1
mask = ~((data_tree_8[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_tree_8[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)
data_clean = data_tree_8[mask].reset_index(drop=True)

print("Tamaño original:", data_tree_8.shape)
print("Tamaño sin outliers:", data_clean.shape)

# 2. Separar X e y
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Definir el escalador y las columnas numéricas (excluyendo 'Gender')
scaler = StandardScaler()
numeric_cols = [col for col in X.columns if col not in ['Gender']]


# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]  
resultados = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=========== 🧠 CASO DE PRUEBA {i} (random_state={seed}) ===========")

    # 1. Dividir datos (80/20 estratificado con el seed actual)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 2. Aplicar Escalado (fit/transform en train, transform en test)
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])


    # 3. Entrenamiento y Predicción de Modelos (con class_weight='balanced')

    # --- Modelo 1: Gini + Balanced ---
    modelo_gini = DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        random_state=seed,
        class_weight='balanced'
    )
    modelo_gini.fit(X_train_scaled, y_train)
    y_pred_gini = modelo_gini.predict(X_test_scaled)

    # --- Modelo 2: Entropía + Balanced ---
    modelo_entropy = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        random_state=seed,
        class_weight='balanced'
    )
    modelo_entropy.fit(X_train_scaled, y_train)
    y_pred_entropy = modelo_entropy.predict(X_test_scaled)

    # --- Modelo 3: Entropía con poda + Balanced ---
    modelo_entropy_pruned = DecisionTreeClassifier(
        criterion='entropy',
        max_depth=5,
        min_samples_split=5, # Poda
        random_state=seed,
        class_weight='balanced'
    )
    modelo_entropy_pruned.fit(X_train_scaled, y_train)
    y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

    # 4. Calcular métricas
    modelos = {
        "Gini_Clean_Scaled_Balanced": y_pred_gini,
        "Entropía_Clean_Scaled_Balanced": y_pred_entropy,
        "Entropía_Podado_Clean_Scaled_Balanced": y_pred_entropy_pruned
    }

    for nombre, y_pred in modelos.items():
        # Calcular métricas (usando zero_division=0 por robustez)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Caso": i,
            "Random State": seed,
            "Modelo": nombre,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1
        })

        print(f"\nModelo: {nombre}")
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

=========== 🧠 CASO DE PRUEBA 1 (random_state=111) ===========

Modelo: Gini_Clean_Scaled_Balanced
Accuracy: 0.7745, Precision: 0.7935, Recall: 0.7745, F1-Score: 0.7710

Modelo: Entropía_Clean_Scaled_Balanced
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

Modelo: Entropía_Podado_Clean_Scaled_Balanced
Accuracy: 0.8042, Precision: 0.8154, Recall: 0.8042, F1-Score: 0.8054

=========== 🧠 CASO DE PRUEBA 2 (random_state=222) ===========

Modelo: Gini_Clean_Scaled_Balanced
Accuracy: 0.7933, Precision: 0.8097, Recall: 0.7933, F1-Score: 0.7919

Modelo: Entropía_Clean_Scaled_Balanced
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

Modelo: Entropía_Podado_Clean_Scaled_Balanced
Accuracy: 0.7982, Precision: 0.8104, Recall: 0.7982, F1-Score: 0.8004

=========== 🧠 CASO DE PRUEBA 3 (random_state=333) ===========

Modelo: Gini_Clean_Scaled_Balanced
Accuracy: 0.7943, Precision: 0.7994, Recall: 0.79

In [18]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

# K Vecinos Más Cercanos:

#### 1. KNN - CC:SI - ED:NO - Outliers:NO - Balanceo: NO

In [40]:
# ================================================================
# 📂 Preparación de los datos
# ================================================================
data_knn_1 = data.copy()

X = data_knn_1.drop("Workout_Type", axis=1)
y = data_knn_1["Workout_Type"]

# Definición de la función de evaluación
def evaluar_modelo(X_train, X_test, y_train, y_test, metric_name, k_value, seed):
    """Entrena y evalúa un modelo KNN."""
    knn = KNeighborsClassifier(n_neighbors=k_value, metric=metric_name)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

    return {
        'Random State': seed,
        'Métrica': metric_name,
        'k': k_value,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    }

# ================================================================
# 🔁 Tres muestras
# ================================================================
random_states = [111, 222, 333]
k_range = range(1, 100)
resultados_finales = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=================================================")
    print(f"🧠 CASO DE PRUEBA {i} (random_state={seed})")
    print(f"=================================================")

    # División de datos (80% entrenamiento, 20% prueba)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 🔎 Búsqueda de mejores K para el split actual
    accuracies_euclidean = []
    accuracies_manhattan = []

    for metric in ['euclidean', 'manhattan']:
        for k in k_range:
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            
            if metric == 'euclidean':
                accuracies_euclidean.append(acc)
            else:
                accuracies_manhattan.append(acc)

    # Encontrar el mejor K
    best_k_euclidean = k_range[accuracies_euclidean.index(max(accuracies_euclidean))]
    best_k_manhattan = k_range[accuracies_manhattan.index(max(accuracies_manhattan))]

    print(f"🔹 Mejor K (Euclidiana): {best_k_euclidean} | Max Acc: {max(accuracies_euclidean):.4f}")
    print(f"🔹 Mejor K (Manhattan): {best_k_manhattan} | Max Acc: {max(accuracies_manhattan):.4f}")

    # 🧠 Evaluación final con los K óptimos
    print("\n--- Evaluación Final ---")
    
    # Evaluar Euclidiana
    print(f"-> Evaluación Euclidiana (k={best_k_euclidean}):")
    resultados_finales.append(
        evaluar_modelo(X_train, X_test, y_train, y_test, 'euclidean', best_k_euclidean, seed)
    )
    
    # Evaluar Manhattan
    print(f"-> Evaluación Manhattan (k={best_k_manhattan}):")
    resultados_finales.append(
        evaluar_modelo(X_train, X_test, y_train, y_test, 'manhattan', best_k_manhattan, seed)
    )


🧠 CASO DE PRUEBA 1 (random_state=111)
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7365
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.7913

--- Evaluación Final ---
-> Evaluación Euclidiana (k=1):
Accuracy: 0.7365, Precision: 0.7362, Recall: 0.7365, F1-Score: 0.7362
-> Evaluación Manhattan (k=1):
Accuracy: 0.7913, Precision: 0.7919, Recall: 0.7913, F1-Score: 0.7916

🧠 CASO DE PRUEBA 2 (random_state=222)
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7423
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.7981

--- Evaluación Final ---
-> Evaluación Euclidiana (k=1):
Accuracy: 0.7423, Precision: 0.7452, Recall: 0.7423, F1-Score: 0.7429
-> Evaluación Manhattan (k=1):
Accuracy: 0.7981, Precision: 0.8021, Recall: 0.7981, F1-Score: 0.7988

🧠 CASO DE PRUEBA 3 (random_state=333)
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7596
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.8115

--- Evaluación Final ---
-> Evaluación Euclidiana (k=1):
Accuracy: 0.7596, Precision: 0.7602, Recall: 0.7596, F1-Score: 0.7596
-> Evaluación Manhattan (k=1):
Accur

In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la gráfica de knn con el modelo entrenado

#### 2. KNN - CC:SI - ED:NO - Outliers:NO - Balanceo: SI

In [41]:
# ================================================================
# 📂 Preparación de los datos
# ================================================================
data_knn_2 = data.copy()

X = data_knn_2.drop("Workout_Type", axis=1)
y = data_knn_2["Workout_Type"]

# Definición de la función de evaluación
def evaluar_modelo(X_train, X_test, y_train, y_test, metric_name, k_value, seed):
    """Entrena y evalúa un modelo KNN usando weights='distance'."""
    # weights='distance' prioriza los vecinos más cercanos, actuando como un balanceo ponderado.
    knn = KNeighborsClassifier(n_neighbors=k_value, metric=metric_name, weights='distance')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")

    return {
        'Random State': seed,
        'Métrica': metric_name,
        'k': k_value,
        'Weights': 'distance',
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    }

# ================================================================
# 🔁 Tres muestras 
# ================================================================
random_states = [111, 222, 333]
k_range = range(1, 100)
resultados_finales = []

for i, seed in enumerate(random_states, start=1):
    print(f"\n=================================================")
    print(f"🧠 CASO DE PRUEBA {i} (random_state={seed}) - Balanced")
    print(f"=================================================")

    # División de datos (80% entrenamiento, 20% prueba)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed, stratify=y
    )

    # 🔎 Búsqueda de mejores K para el split actual (usando weights='distance')
    accuracies_euclidean = []
    accuracies_manhattan = []

    for metric in ['euclidean', 'manhattan']:
        for k in k_range:
            # Importante: Usamos weights='distance' en la búsqueda de K
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric, weights='distance')
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            
            if metric == 'euclidean':
                accuracies_euclidean.append(acc)
            else:
                accuracies_manhattan.append(acc)

    # Encontrar el mejor K
    best_k_euclidean = k_range[accuracies_euclidean.index(max(accuracies_euclidean))]
    best_k_manhattan = k_range[accuracies_manhattan.index(max(accuracies_manhattan))]

    print(f"🔹 Mejor K (Euclidiana): {best_k_euclidean} | Max Acc: {max(accuracies_euclidean):.4f}")
    print(f"🔹 Mejor K (Manhattan): {best_k_manhattan} | Max Acc: {max(accuracies_manhattan):.4f}")

    # 🧠 Evaluación final con los K óptimos
    print("\n--- Evaluación Final ---")
    
    # Evaluar Euclidiana
    print(f"-> Evaluación Euclidiana (k={best_k_euclidean}, weights='distance'):")
    resultados_finales.append(
        evaluar_modelo(X_train, X_test, y_train, y_test, 'euclidean', best_k_euclidean, seed)
    )
    
    # Evaluar Manhattan
    print(f"-> Evaluación Manhattan (k={best_k_manhattan}, weights='distance'):")
    resultados_finales.append(
        evaluar_modelo(X_train, X_test, y_train, y_test, 'manhattan', best_k_manhattan, seed)
    )


🧠 CASO DE PRUEBA 1 (random_state=111) - Balanced
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7365
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.7913

--- Evaluación Final ---
-> Evaluación Euclidiana (k=1, weights='distance'):
Accuracy: 0.7365, Precision: 0.7362, Recall: 0.7365, F1-Score: 0.7362
-> Evaluación Manhattan (k=1, weights='distance'):
Accuracy: 0.7913, Precision: 0.7919, Recall: 0.7913, F1-Score: 0.7916

🧠 CASO DE PRUEBA 2 (random_state=222) - Balanced
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7423
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.7981

--- Evaluación Final ---
-> Evaluación Euclidiana (k=1, weights='distance'):
Accuracy: 0.7423, Precision: 0.7452, Recall: 0.7423, F1-Score: 0.7429
-> Evaluación Manhattan (k=1, weights='distance'):
Accuracy: 0.7981, Precision: 0.8021, Recall: 0.7981, F1-Score: 0.7988

🧠 CASO DE PRUEBA 3 (random_state=333) - Balanced
🔹 Mejor K (Euclidiana): 1 | Max Acc: 0.7596
🔹 Mejor K (Manhattan): 1 | Max Acc: 0.8115

--- Evaluación Final ---
-> Evaluación Euclidiana 

In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la gráfica de knn con el modelo entrenado

#### 3. KNN - CC:SI - ED:NO - Outliers:SI - Balanceo: NO